In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler , Binarizer 
from sklearn.model_selection import train_test_split , GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_curve , roc_curve

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as ms
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# XGBoost
- 트리 기반의 알고지름
- 뛰어난 예측 성능
- 빠른 수행시간
- 병렬 (멀티) 같이 수행가능

- 일반 파라미터 
- 스래드의 개수나 slient모드의 선택을 위한 파라미터

In [ ]:
from sklearn.datasets import load_breast_cancer
dataset=load_breast_cancer()
# dataset.keys()
features=dataset.data
#featurs
label=dataset.target # 0,1로 구성 이진분류
#label

cancer_df=pd.DataFrame(data=features,columns=dataset.feature_names)
cancer_df['target']=label
cancer_df.head()

In [ ]:
# 클래스 분포비율 확인
cancer_df['target'].value_counts()

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(features,
                                               label,
                                               test_size=0.2,
                                               random_state=100)

- sklearn XGBoost 적용

In [ ]:
# 학습을 위해 분류기 생성
from xgboost import XGBRFClassifier

In [ ]:
sklearn_xgboost_model=XGBRFClassifier(n_estimators=400,
                                      learning_rate=0.1,
                                      max_depth=3) # 0.3을 디폴트
sklearn_xgboost_model.fit(X_train,y_train)

In [ ]:
y_pred=sklearn_xgboost_model.predict(X_test)

In [ ]:
# 하이퍼파라미터 튜닝을 이용한 예측 정확도를 실습해 보세요!!!
# 혼동행렬, 정확도, 정밀도, 재현율, F1 , AUC 확인까지

def classifier_eval(y_test , y_pred) :
    print('오차행렬 : ' , confusion_matrix(y_test, y_pred))
    print('정확도   : ' , accuracy_score(y_test, y_pred))
    print('정밀도   : ' , precision_score(y_test, y_pred))
    print('재현율   : ' , recall_score(y_test, y_pred))
    print('F1       : ' , f1_score(y_test, y_pred))
    print('AUC      : ' , roc_auc_score(y_test, y_pred))


In [ ]:
classifier_eval(y_test,y_pred)

In [ ]:
# 튜닝
# logloss-> 오류에 대한 평가성능지표 리턴 함수
sklearn_xgboost_model=XGBRFClassifier(n_estimators=400,
                                      learning=0.1,
                                      max_depth=3)
sklearn_xgboost_model.fit(X_train,y_train,
                          early_stopping_rounds=100,
                          eval_metric='logloss',
                          eval_set=[(X_test,y_test)], # [(X_test,y_test)] 예측할때 쓰는 학습데이터
                          verbose=True)

In [ ]:
y_pred145=sklearn_xgboost_model.predict(X_test)
classifier_eval(y_test,y_pred145)

In [ ]:
# 피처 중요도 시각화
from xgboost import plot_importance
plot_importance(sklearn_xgboost_model)

# stacking 
- 여러 모델을 활용해서 각각 예측 결과를 도출한 뒤 그 예측 결과를 결합해 최종 예측 결과를 만들어내는것

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# 개별 분류기 생성
knn_clf=KNeighborsClassifier(n_neighbors=4)
rf_clf=RandomForestClassifier(n_estimators=100,random_state=100)
dt_clf=DecisionTreeClassifier()
ada_clf=AdaBoostClassifier(n_estimators=100)

In [ ]:
# 개별 모델에 대한 학습
knn_clf.fit(X_train,y_train)
rf_clf.fit(X_train,y_train)
dt_clf.fit(X_train,y_train)
ada_clf.fit(X_train,y_train)

In [ ]:
# 개별 모델에 대한 예측을 수행
knn_pred=knn_clf.predict(X_test)
rf_pred=rf_clf.predict(X_test)
dt_pred=dt_clf.predict(X_test)
ada_pred=ada_clf.predict(X_test)

In [ ]:
classifier_eval(y_test,knn_pred)
classifier_eval(y_test,rf_pred)
classifier_eval(y_test,dt_pred)
classifier_eval(y_test,ada_pred)

In [ ]:
# 학습을 마친 모델의 예측결과를 합쳐서 최종 예측
merge_pred=np.array([knn_pred,rf_pred,dt_pred,ada_pred])
merge_pred

In [ ]:
merge_pred=np.transpose(merge_pred)
merge_pred

In [ ]:
# 최종분류기 선택 MetaModel 만들어보자
lr_clf=LogisticRegression()
lr_clf.fit(merge_pred,y_test)
final_pred=lr_clf.predict(merge_pred)
classifier_eval(y_test,final_pred)

In [ ]:
# 과적합을 보완하기위해서 교차검증을 추가해야한다.
# CV세트를 기반으로 코드를 변경
# 회귀모델의 평가지료로 활용-> MAE(Mean of Absolute Errors)
# 실제값과 예측 값의 차이를 절대값으로 변환된 평균

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [ ]:
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    
    print(model.__class__.__name__ , ' model 시작 ')
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean


In [ ]:
# CV 적용 모델의 데이터 세트 리턴
knn_train,knn_test=get_stacking_base_datasets(knn_clf,X_train,y_train,X_test,5)
rf_train,rf_test=get_stacking_base_datasets(rf_clf,X_train,y_train,X_test,5)
dt_train,dt_test=get_stacking_base_datasets(dt_clf,X_train,y_train,X_test,5)
ada_train,ada_test=get_stacking_base_datasets(ada_clf,X_train,y_train,X_test,5)

In [ ]:
stacking_train=np.concatenate((knn_train,rf_train,dt_train,ada_train),axis=1)
#stacking_train
stacking_test=np.concatenate((knn_test,rf_test,dt_test,ada_test),axis=1)
#stacking_test

In [ ]:
#메타 모델 학습
lr_clf.fit(stacking_train,y_train) #stacking_train는 features 을 가지고있다.
stacking_pred=lr_clf.predict(stacking_test)
classifier_eval(y_test,stacking_pred)